# Densely Connected Networks (DenseNet)

ResNet significantly changed the view of how to parametrize the functions in deep networks. DenseNet is to some extent the logical extension of this. To understand how to arrive at it, let us take a small detour to theory. Recall the Taylor expansion for functions. For scalars it can be written as

$$f(x) = f(0) + f'(x) x + \frac{1}{2} f''(x) x^2 + \frac{1}{6} f'''(x) x^3 + o(x^3).$$

## Function Decomposition

The key point is that it decomposes the function into increasingly higher order terms. In a similar vein, ResNet decomposes functions into

$$f(\mathbf{x}) = \mathbf{x} + g(\mathbf{x}).$$

That is, ResNet decomposes $f$ into a simple linear term and a more complex
nonlinear one. What if we want to go beyond two terms? A solution was proposed
by :cite:`Huang.Liu.Van-Der-Maaten.ea.2017` in the form of
DenseNet, an architecture that reported record performance on the ImageNet
dataset.

![The main difference between ResNet (left) and DenseNet (right) in cross-layer connections: use of addition and use of concatenation. ](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/densenet-block.svg)

:label:`fig_densenet_block`


As shown in :numref:`fig_densenet_block`, the key difference between ResNet and DenseNet is that in the latter case outputs are *concatenated* rather than added. As a result we perform a mapping from $\mathbf{x}$ to its values after applying an increasingly complex sequence of functions.

$$\mathbf{x} \to \left[\mathbf{x}, f_1(\mathbf{x}), f_2(\mathbf{x}, f_1(\mathbf{x})), f_3(\mathbf{x}, f_1(\mathbf{x}), f_2(\mathbf{x}, f_1(\mathbf{x})), \ldots\right].$$

In the end, all these functions are combined in an MLP to reduce the number of features again. In terms of implementation this is quite simple---rather than adding terms, we concatenate them. The name DenseNet arises from the fact that the dependency graph between variables becomes quite dense. The last layer of such a chain is densely connected to all previous layers. The main components that compose a DenseNet are dense blocks and transition layers. The former defines how the inputs and outputs are concatenated, while the latter controls the number of channels so that it is not too large. The dense connections are shown in :numref:`fig_densenet`.

![Dense connections in DenseNet](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/densenet.svg)

:label:`fig_densenet`



## Dense Blocks

DenseNet uses the modified "batch normalization, activation, and convolution"
architecture of ResNet (see the exercise in :numref:`sec_resnet`).
First, we implement this architecture in the
`conv_block` function.

In [1]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import jp.live.ugai.d2j.util.Training

In [2]:
import ai.djl.basicdataset.cv.classification.*;
import ai.djl.metric.*;
// import org.apache.commons.lang3.ArrayUtils;

In [3]:
fun convBlock(numChannels: Int): SequentialBlock {
    return SequentialBlock()
        .add(BatchNorm.builder().build())
        .add(Activation::relu)
        .add(
            Conv2d.builder()
                .setFilters(numChannels)
                .setKernelShape(Shape(3, 3))
                .optPadding(Shape(1, 1))
                .optStride(Shape(1, 1))
                .build()
        )
}

A dense block consists of multiple `convBlock` units, each using the same number of output channels. In the forward computation, however, we concatenate the input and output of each block on the channel dimension.

In [4]:
class DenseBlock(numConvs: Int, numChannels: Int) : AbstractBlock(VERSION) {
    var net = SequentialBlock()

    init {
        for (i in 0 until numConvs) {
            net.add(addChildBlock("denseBlock$i", convBlock(numChannels)))
        }
    }

    override fun toString(): String {
        return "DenseBlock()"
    }

    override fun forwardInternal(
        parameterStore: ParameterStore,
        X: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        var X = X
        var Y: NDArray
        for (block in net.children.values()) {
            Y = block.forward(parameterStore, X, training).singletonOrThrow()
            X = NDList(NDArrays.concat(NDList(X.singletonOrThrow(), Y), 1))
        }
        return X
    }

    override fun getOutputShapes(inputs: Array<Shape>): Array<Shape> {
        val shapesX: Array<Shape> = inputs
        for (block in net.children.values()) {
            val shapesY: Array<Shape> = block.getOutputShapes(shapesX)
            shapesX[0] = Shape(
                shapesX[0].get(0),
                shapesY[0].get(1) + shapesX[0].get(1),
                shapesX[0].get(2),
                shapesX[0].get(3)
            )
        }
        return shapesX
    }

    override fun initializeChildBlocks(manager: NDManager?, dataType: DataType?, vararg inputShapes: Shape) {
        var shapesX: Shape = inputShapes[0]
        for (block in net.children.values()) {
            block.initialize(manager, DataType.FLOAT32, shapesX)
            val shapesY: Array<Shape> = block.getOutputShapes(arrayOf(shapesX))
            shapesX = Shape(
                shapesX.get(0),
                shapesY[0].get(1) + shapesX.get(1),
                shapesX.get(2),
                shapesX.get(3)
            )
        }
    }

    companion object {
        private const val VERSION: Byte = 1
    }
}


In the following example, we define a convolution block (`DenseBlock`) with two blocks of 10 output channels. When using an input with 3 channels, we will get an output with the $3+2\times 10=23$ channels. The number of convolution block channels controls the increase in the number of output channels relative to the number of input channels. This is also referred to as the growth rate.

In [5]:
    val manager = NDManager.newBaseManager()
    var block = SequentialBlock().add(DenseBlock(2, 10))

    val X = manager.randomUniform(0.0f, 1.0f, Shape(4, 3, 8, 8))

    block.initialize(manager, DataType.FLOAT32, X.shape)

    val parameterStore = ParameterStore(manager, true)

    var currentShape = arrayOf<Shape>(X.shape)
    for (child in block.children.values()) {
        currentShape = child.getOutputShapes(currentShape)
    }

    println(currentShape[0])

(4, 23, 8, 8)


## Transition Layers

Since each dense block will increase the number of channels, adding too many of them will lead to an excessively complex model. A transition layer is used to control the complexity of the model. It reduces the number of channels by using the $1\times 1$ convolutional layer and halves the height and width of the average pooling layer with a stride of 2, further reducing the complexity of the model.

In [6]:
fun transitionBlock(numChannels: Int): SequentialBlock {
    return SequentialBlock()
        .add(BatchNorm.builder().build())
        .add { arrays: NDList? -> Activation.relu(arrays) }
        .add(
            Conv2d.builder()
                .setFilters(numChannels)
                .setKernelShape(Shape(1, 1))
                .optStride(Shape(1, 1))
                .build()
        )
        .add(Pool.avgPool2dBlock(Shape(2, 2), Shape(2, 2)))
}

Apply a transition layer with 10 channels to the output of the dense block in the previous example.  This reduces the number of output channels to 10, and halves the height and width.

In [7]:
    block = transitionBlock(10)

    block.initialize(manager, DataType.FLOAT32, currentShape[0])

    for (pair in block.children) {
        currentShape = pair.value.getOutputShapes(currentShape)
    }

    println(currentShape[0])

(4, 10, 4, 4)


## DenseNet Model

Next, we will construct a DenseNet model. DenseNet first uses the same single convolutional layer and maximum pooling layer as ResNet.

In [8]:
    val net = SequentialBlock()
        .add(
            Conv2d.builder()
                .setFilters(64)
                .setKernelShape(Shape(7, 7))
                .optStride(Shape(2, 2))
                .optPadding(Shape(3, 3))
                .build()
        )
        .add(BatchNorm.builder().build())
        .add { arrays: NDList? -> Activation.relu(arrays) }
        .add(Pool.maxPool2dBlock(Shape(3, 3), Shape(2, 2), Shape(1, 1)))

Then, similar to the four residual blocks that ResNet uses, DenseNet uses four dense blocks. Similar to ResNet, we can set the number of convolutional layers used in each dense block. Here, we set it to 4, consistent with the ResNet-18 in the previous section. Furthermore, we set the number of channels (i.e., growth rate) for the convolutional layers in the dense block to 32, so 128 channels will be added to each dense block.

In ResNet, the height and width are reduced between each module by a residual block with a stride of 2. Here, we use the transition layer to halve the height and width and halve the number of channels.

In [9]:
    var numChannels: Int = 64
    val growthRate = 32

    val numConvsInDenseBlocks = intArrayOf(4, 4, 4, 4)

    for (index in numConvsInDenseBlocks.indices) {
        val numConvs = numConvsInDenseBlocks[index]
        net.add(DenseBlock(numConvs, growthRate))
        numChannels += numConvs * growthRate
        if (index != numConvsInDenseBlocks.size - 1) {
            numChannels = numChannels / 2
            net.add(transitionBlock(numChannels))
        }
    }

Similar to ResNet, a global pooling layer and fully connected layer are connected at the end to produce the output.

In [10]:
    net
        .add(BatchNorm.builder().build())
        .add(Activation::relu)
        .add(Pool.globalAvgPool2dBlock())
        .add(Linear.builder().setUnits(10).build())

    println(net)

SequentialBlock {
	Conv2d
	BatchNorm
	LambdaBlock
	maxPool2d
	DenseBlock {
		denseBlock0 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
		denseBlock1 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
		denseBlock2 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
		denseBlock3 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
	}
	SequentialBlock {
		BatchNorm
		LambdaBlock
		Conv2d
		avgPool2d
	}
	DenseBlock {
		denseBlock0 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
		denseBlock1 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
		denseBlock2 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
		denseBlock3 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
	}
	SequentialBlock {
		BatchNorm
		LambdaBlock
		Conv2d
		avgPool2d
	}
	DenseBlock {
		denseBlock0 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
		denseBlock1 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
		denseBlock2 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
		denseBlock3 {
			BatchNorm
			LambdaBlock
			Conv2d
		}
	}
	SequentialBlock {
		BatchNorm
		LambdaBlock
		Conv2d


## Data Acquisition and Training

Since we are using a deeper network here, in this section, we will reduce the input height and width from 224 to 96 to simplify the computation.

In [13]:
    val batchSize = 256
    val lr = 0.1f
    val numEpochs = Integer.getInteger("MAX_EPOCH", 10)

//    var trainLoss: DoubleArray
//    var testAccuracy: DoubleArray
//    var trainAccuracy: DoubleArray

    val epochCount = IntArray(numEpochs) { it + 1 }

    val trainIter = FashionMnist.builder()
        .addTransform(Resize(96))
        .addTransform(ToTensor())
        .optUsage(Dataset.Usage.TRAIN)
        .setSampling(batchSize, true)
        .optLimit(java.lang.Long.getLong("DATASET_LIMIT", Long.MAX_VALUE))
        .build()

    val testIter = FashionMnist.builder()
        .addTransform(Resize(96))
        .addTransform(ToTensor())
        .optUsage(Dataset.Usage.TEST)
        .setSampling(batchSize, true)
        .optLimit(java.lang.Long.getLong("DATASET_LIMIT", Long.MAX_VALUE))
        .build()

    trainIter.prepare()
    testIter.prepare()

    val model = Model.newInstance("cnn")
    model.block = net

    val loss: Loss = Loss.softmaxCrossEntropyLoss()

    val lrt: Tracker = Tracker.fixed(lr)
    val sgd: Optimizer = Optimizer.sgd().setLearningRateTracker(lrt).build()

    val config = DefaultTrainingConfig(loss).optOptimizer(sgd) // Optimizer (loss function)
        .addEvaluator(Accuracy()) // Model Accuracy
        .addTrainingListeners(*TrainingListener.Defaults.logging()) // Logging

    val trainer: Trainer = model.newTrainer(config)
    trainer.initialize(Shape(1, 1, 96, 96))

    val evaluatorMetrics: MutableMap<String, DoubleArray> = mutableMapOf()
    val avgTrainTimePerEpoch = Training.trainingChapter6(trainIter, testIter, numEpochs, trainer, evaluatorMetrics)

Training:    100% |████████████████████████████████████████| Accuracy: 0.83, SoftmaxCrossEntropyLoss: 0.51
Validating:  100% |████████████████████████████████████████|
Training:     44% |██████████████████                      | Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.31

The execution was interrupted

In [14]:
    val trainLoss = evaluatorMetrics.get("train_epoch_SoftmaxCrossEntropyLoss")
    val trainAccuracy = evaluatorMetrics.get("train_epoch_Accuracy")
    val testAccuracy = evaluatorMetrics.get("validate_epoch_Accuracy")

    print("loss %.3f,".format(trainLoss!![numEpochs - 1]))
    print(" train acc %.3f,".format(trainAccuracy!![numEpochs - 1]))
    print(" test acc %.3f\n".format(testAccuracy!![numEpochs - 1]))
    println("%.1f examples/sec".format(trainIter.size() / (avgTrainTimePerEpoch / Math.pow(10.0, 9.0))))

null
java.lang.NullPointerException
	at Line_56.<init>(Line_56.jupyter-kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:105)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

![Contour Gradient Descent.](https://d2l-java-resources.s3.amazonaws.com/img/chapter_convolution-modern-cnn-denseNet.png)

In [15]:
val trainLossLabel =  Array<String>(trainLoss!!.size) { "train loss" }
val trainAccLabel = Array<String>(trainLoss!!.size) { "train acc" }
val testAccLabel = Array<String>(trainLoss!!.size) { "test acc" }
val data = mapOf<String, Any>(
      "label" to trainLossLabel + trainAccLabel + testAccLabel,
      "epoch" to epochCount + epochCount + epochCount,
      "metrics" to trainLoss!! + trainAccuracy!! + testAccuracy!!
)

var plot = letsPlot(data)
plot += geomLine { x = "epoch" ; y = "metrics" ; color = "label"}
plot + ggsize(700, 500)

null
java.lang.NullPointerException
	at Line_58.<init>(Line_58.jupyter-kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:105)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

## Summary

* In terms of cross-layer connections, unlike ResNet, where inputs and outputs are added together, DenseNet concatenates inputs and outputs on the channel dimension.
* The main units that compose DenseNet are dense blocks and transition layers.
* We need to keep the dimensionality under control when composing the network by adding transition layers that shrink the number of channels again.

## Exercises

1. Why do we use average pooling rather than max-pooling in the transition layer?
1. One of the advantages mentioned in the DenseNet paper is that its model parameters are smaller than those of ResNet. Why is this the case?
1. One problem for which DenseNet has been criticized is its high memory consumption.
    * Is this really the case? Try to change the input shape to $224\times 224$ to see the actual (GPU) memory consumption.
    * Can you think of an alternative means of reducing the memory consumption? How would you need to change the framework?
1. Implement the various DenseNet versions presented in Table 1 of :cite:`Huang.Liu.Van-Der-Maaten.ea.2017`.
1. Why do we not need to concatenate terms if we are just interested in $\mathbf{x}$ and $f(\mathbf{x})$ for ResNet? Why do we need this for more than two layers in DenseNet?
1. Design a DenseNet for fully connected networks and apply it to the Housing Price prediction task.